In [9]:
import sys    
import os    
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
import torch as T
import torch.nn as nn 

In [10]:
test_critic= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='tb', learning_rate=0.02)
test_critic.save_checkpoint()
test_state= T.tensor([[-1.4355, -0.7806,  0.3042,  1.1601]])
test_action= T.tensor([[0.8233, 0.8126]])
# for target_param, param in zip(test_critic.parameters(), test_critic.network.parameters()):
#     print(target_param-param)

... saving checkpoint ...


RuntimeError: Parent directory /Users/niyi/Documents/GitHub/Optimal-Control/Data/Temp/critic_data does not exist.

In [ ]:
test_critic(test_state, test_action)

tensor([[-0.8415]], grad_fn=<AddmmBackward0>)

In [ ]:
test_actor= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), 'bt', 0.02)
test_critic.save_checkpoint()
# for target_param in test_actor.parameters():
#     print(target_param)
# Nt.load_checkpoint()


... saving checkpoint ...


In [ ]:
test_actor(test_state)

tensor([[0.3046, 0.5316]], grad_fn=<AddmmBackward0>)